In [ ]:
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps

In [ ]:
def initialize_producer(bootstrap_servers):
    """Initialize Kafka producer."""
    return KafkaProducer(
        bootstrap_servers=bootstrap_servers,
        value_serializer=lambda x: dumps(x).encode('utf-8')
    )

In [ ]:
def send_initial_message(producer, topic, message):
    producer.send(topic, value=message)

def read_data(file_path):
    return pd.read_csv(file_path)

def produce_messages(producer, topic, df, delay):
    while True:
        dict_stock = df.sample(1).to_dict(orient="records")[0]
        producer.send(topic, value=dict_stock)
        sleep(delay)

In [ ]:
bootstrap_servers = ['your_ip_here:9092']  # Replace with your IP
topic = 'demo_test'
csv_file_path = "indexProcessed.csv"
initial_message = {'lastname': 'Vadla'}
delay = 1  # delay in seconds

producer = initialize_producer(bootstrap_servers)
send_initial_message(producer, topic, initial_message)
    
df = read_data(csv_file_path)
    
try:
    produce_messages(producer, topic, df, delay)
except KeyboardInterrupt:
    print("Shutting down producer.")
finally:
    producer.flush()  # Ensure all messages are sent